In [1]:
import pandas as pd

In [2]:
import random
n = 216930 
s = 10000
skip = [0].append(sorted(random.sample(range(n),n-s)))
data = pd.read_csv('JEOPARDY_CSV.csv', header=0, skiprows=lambda i: i>0 and random.random() > 0.02)
data.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$1000,This Asian political party was founded in 1885...,the Congress Party
1,4680,2004-12-31,Double Jeopardy!,THAT OLD-TIME RELIGION,$1200,Philadelphia got its start as a colony for thi...,the Quakers
2,5957,2010-07-06,Double Jeopardy!,KIDS IN SPORTS,$1600,11-year-old Ashlyn White won a 2009 U.S. youth...,judo
3,3751,2000-12-18,Double Jeopardy!,ANDY WARHOL,$400,Warhol went against his Capitalist tendencies ...,Mao Tse-tung
4,3673,2000-07-19,Jeopardy!,1994 FILMS,$100,Quentin Tarantino directed this film & also ha...,Pulp Fiction


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4236 entries, 0 to 4235
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Show Number  4236 non-null   int64 
 1    Air Date    4236 non-null   object
 2    Round       4236 non-null   object
 3    Category    4236 non-null   object
 4    Value       4236 non-null   object
 5    Question    4236 non-null   object
 6    Answer      4236 non-null   object
dtypes: int64(1), object(6)
memory usage: 231.8+ KB


In [4]:
X = data[[' Question']]
X.head()

,Question
0,This Asian political party was founded in 1885...
1,Philadelphia got its start as a colony for thi...
2,11-year-old Ashlyn White won a 2009 U.S. youth...
3,Warhol went against his Capitalist tendencies ...
4,Quentin Tarantino directed this film & also ha...


In [5]:
with open('stopwords.txt', encoding='utf8') as stopwords_file:
    stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n', '') for line in stopwords]
stopwords

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 "can't",
 'cannot',
 'could',
 "couldn't",
 'did',
 "didn't",
 'do',
 'does',
 "doesn't",
 'doing',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 "hadn't",
 'has',
 "hasn't",
 'have',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 "here's",
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 "how's",
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 "let's",
 'me',
 'more',
 'most',
 "mustn't",
 'my',
 'myself',
 'no',
 'nor',
 'not',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'ought',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'same',
 "shan't",
 'she',
 "she'd",
 "she'll",
 "she's",
 'should',
 "s

In [6]:
import re
s = 'this. is: a string32 2 333?'
s = re.sub('[^\w\s]', '', s)
s

'this is a string32 2 333'

In [7]:
from nltk import word_tokenize
import nltk
import re
nltk.download('punkt')
dataset = pd.DataFrame(columns=['title_body'])
for index, row in X.iterrows():
    title_body_tokenized = word_tokenize(row[' Question'])
    title_body_tokenized_filtered = [w.lower() for w in title_body_tokenized if not w.lower() in stopwords]
    #     title_body_tokenized_filtered_stemmed = [stemmer.stem(w) for w in title_body_tokenized_filtered]
    s = re.sub('[^\w\s]', '', ' '.join(title_body_tokenized_filtered))
    s = re.sub("\d+", "", s)
    dataset.loc[index] = {
        'title_body': s,
    }

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kavia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
dataset

,title_body
0,asian political party founded indian national
1,philadelphia start colony religious william penn
2,yearold ashlyn white us youth title martial a...
3,warhol capitalist tendencies portrait
4,quentin tarantino directed film bit role jimm...
...,...
4231,adjective lot uninhabited house refers estate ...
4232,s term buddhist temple thailand cambodia angkor
4233,memoirs administration eve rebellion published
4234,abstemious adjudicator


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vectorizer = TfidfVectorizer()
vectorizer.fit(dataset['title_body'])

TfidfVectorizer()

In [11]:
X = vectorizer.transform(dataset['title_body'])

In [12]:
X

<4236x11112 sparse matrix of type '<class 'numpy.float64'>'
	with 27905 stored elements in Compressed Sparse Row format>

In [13]:
import umap
reducer = umap.UMAP(n_components=1000)

In [14]:
X_reduced = reducer.fit_transform(X)

In [15]:
import numpy as np
np.shape(X_reduced)

In [16]:
from sklearn.cluster import MeanShift, SpectralClustering

In [20]:
msh = SpectralClustering()
msh.fit(X_reduced)

SpectralClustering()

In [21]:
from sklearn.metrics import silhouette_score

In [22]:
silhouette_score(X, msh.labels_)

0.006879829887453635

In [23]:
dataset['Cluster'] = msh.labels_

In [28]:
dataset[dataset['Cluster'] == 2]

,title_body,Cluster
16,owain glyndwr died circa native country cla...,2
23,capital french protectorate tonkin s capital ...,2
161,country land shamrock,2
192,gi blues set european country elvis served ...,2
212,truce suspending war divided country signed p...,2
...,...,...
4079,pig film babe shot farm country,2
4104,country controls eastern tierra del fuego lar...,2
4113,s diamonds including mogul asian country,2
4127,almanac editions us country,2


In [25]:
from collections import Counter

In [29]:
Counter(" ".join(dataset[dataset['Cluster'] == 2]["title_body"]).split(' ')).most_common(20)

[('', 271),
 ('country', 110),
 ('s', 39),
 ('capital', 7),
 ('minister', 7),
 ('king', 6),
 ('prime', 6),
 ('official', 6),
 ('war', 5),
 ('called', 5),
 ('named', 5),
 ('languages', 5),
 ('french', 4),
 ('set', 4),
 ('time', 4),
 ('asian', 4),
 ('germany', 4),
 ('people', 4),
 ('independence', 4),
 ('president', 4)]

In [ ]:
# MasoudKaviani.ir